In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
checkpoint = "HuggingFaceTB/SmolLM-135M-Instruct"

device = "cpu" # for GPU usage or "cpu" for CPU usage
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# for multiple GPUs install accelerate and do `model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")`
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)


/Users/ylow/anaconda3/envs/py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ylow/anaconda3/envs/py311/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/ylow/anaconda3/envs/py311/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <9AF5A39D-A91D-36CA-BAC2-6A31104C2E9C> /Users/ylow/anaconda3/envs/py311/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <D1E1EC61-A5AA-3BBA-94B8-043C751C8A50> /Users/ylow/anaconda3/envs/py311/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your

In [7]:
from datasets import load_dataset

In [8]:
import os
if not os.path.exists('ds/train'):
    train = load_dataset("hakurei/open-instruct-v1", split='train[0%:95%]')
    test = load_dataset("hakurei/open-instruct-v1", split='train[95%:100]')
    def tokenize_function(message):
        m = {}
        m['role'] = 'user'
        m['content'] = message['instruction']
        ret = {}
        prompt = tokenizer.apply_chat_template([m])
        output = tokenizer.encode(message['output'])
        r = prompt + output
        if len(r) >= 2048:
            r = r[:2048]
        r = r[::-1]
        r += [tokenizer.pad_token_id] * (tokenizer.max_len_single_sentence - len(r))
        ret['input_ids'] = r
        ret['labels'] = r
        ret['output'] = message['output']
        return ret


    train = train.map(tokenize_function)
    test = test.map(tokenize_function)
    train = train.shuffle(seed=42)
    train.save_to_disk('ds/train')
    test.save_to_disk('ds/test')


In [2]:
from datasets import load_dataset, load_from_disk
train = load_from_disk('ds/train')
test = load_from_disk('ds/test')

In [3]:
from transformers import TrainingArguments

import numpy as np

import evaluate

metric = evaluate.load("accuracy")

from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", 
                                  eval_strategy="steps",
                                  per_device_train_batch_size=1,
                                  per_device_eval_batch_size=1,
                                  gradient_accumulation_steps=1,
                                  eval_accumulation_steps=1,
                                  eval_steps=100000,
                                  num_train_epochs=1.0)

In [4]:
print(training_args)

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=1,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=100000,
eval_strategy=IntervalStrategy.STEPS,
eval_u

In [5]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=test,
)

In [6]:
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 